<a href="https://colab.research.google.com/github/fellowship/deep-and-wide-bandit/blob/dev/Space_Bandits_Full_Baseline_Kartha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install space-bandits

  Created wheel for space-bandits: filename=space_bandits-0.0.992-cp36-none-any.whl size=17441 sha256=d3802b0651c791d0cdac14e8b21662c23177eed1e4d0187b6cff1ccdfa1042bc
  Stored in directory: /root/.cache/pip/wheels/a7/0c/a7/4bae68f8e665b4c68e7cb7ae7fff88269f3551a4b56b598a42
Successfully built space-bandits


#Importing packages

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd

#Formatting .describe() calls s.t. floats are displayed in non-scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import random
import seaborn as sns
import numpy as np
import json
import pickle as pkl
import warnings
from pprint import pprint
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Importing Supporting Files + Dataset


##Importing dtypes.json

In [4]:
#Set the filepath
dtypes_filepath = '/content/gdrive/MyDrive/Bandit_Project/dtypes.json'

#Load dtypes.json to a dtypes dictionary
with open(dtypes_filepath, mode='r') as dtypes_file:
  dtypes_dict = json.load(dtypes_file)

#Print the keys and values of dtypes_dict to cross-check
print("[INFO] Printing the dtypes dictionary... ")
pprint(dtypes_dict)

[INFO] Printing the dtypes dictionary... 
{'aq_dow': 'object',
 'aq_mo': 'object',
 'aq_period': 'bool',
 'campaign_Brand': 'int64',
 'campaign_Collection': 'int64',
 'campaign_Core': 'int64',
 'campaign_Dedicated': 'int64',
 'campaign_InnovationSpotlight': 'int64',
 'campaign_NewArrivals': 'int64',
 'campaign_Other': 'int64',
 'campaign_ProductSpotlight': 'int64',
 'campaign_Replen': 'int64',
 'campaign_Tops': 'int64',
 'campaign_Trend': 'int64',
 'campaign_id': 'int64',
 'days_subscr': 'int64',
 'discount': 'int64',
 'free_shipping': 'int64',
 'frequency_score': 'int64',
 'has_urgency': 'int64',
 'is_discount_mentioned': 'int64',
 'is_exclusive': 'int64',
 'is_one_for_free': 'int64',
 'message_size': 'int64',
 'opened': 'int64',
 'optimal_action': 'int64',
 'promo': 'int64',
 'recency_score': 'float64',
 'retention_score': 'float64',
 'rev_3dv2': 'float64',
 'reward': 'int64',
 'riid': 'int64',
 'sale': 'int64',
 'sends_since_last_open': 'int64',
 'sent_dayofweek': 'int64',
 'sent_hr

##Importing rolling_statistics.pkl

In [5]:
#Set the filepath
rolling_stats_filepath = '/content/gdrive/MyDrive/Bandit_Project/rolling_statistics.pkl'

#Load rolling_statistics.pkl to a rolling  dictionary
with open(rolling_stats_filepath, mode='rb') as rolling_stats_file:
  rolling_stats = pkl.load(rolling_stats_file)

#Print the keys and values of the dictionary to cross-check
print("[INFO] Printing the rolling_stats dictionary... ")
pprint(rolling_stats)

[INFO] Printing the rolling_stats dictionary... 
{'days_subscr': {'max': 3917,
                 'mean': 1332.471434552341,
                 'min': 0,
                 'std': 822.6802670330655},
 'discount': {'max': 75,
              'mean': 11.300244227332426,
              'min': 0,
              'std': 19.60056341212265},
 'frequency_score': {'max': 163,
                     'mean': 14.977138288600283,
                     'min': 0,
                     'std': 20.754428265423773},
 'message_size': {'max': 354737,
                  'mean': 155242.30496730257,
                  'min': 80267,
                  'std': 36298.15284468782},
 'previous_n_so_far': 88653468,
 'recency_score': {'max': 27.428571428571427,
                   'mean': 1.23904221901564,
                   'min': 0.0,
                   'std': 2.216794122042123},
 'retention_score': {'max': 28.0,
                     'mean': 11.467980895825553,
                     'min': 0.15384615384615385,
                     'st

In [13]:
"""
Code for adding a new variable that stores the sizes of each weekly csv
for train_test_split

#Create a sorted list of path objects & path object names for each of the weekly files
from pathlib import Path
weekly_file_names_generator = Path("/content/gdrive/MyDrive/Bandit_Project").glob('df_20*.csv.gz')
weekly_file_paths_list = sorted(list(weekly_file_names_generator))
weekly_file_names_list = [path_obj.stem for path_obj in weekly_file_paths_list]

#Initialize a dictionary to keep track of the count
weekly_numbers = {}

#Iterate through the weekly file paths list to get length of each dataframe
for path_obj, file_name in zip(weekly_file_paths_list, weekly_file_names_list):

  #Print status update
  pprint(f"[INFO] Working on {file_name}...")

  #Get the length of the dataframe
  df = pd.read_csv(path_obj, header=None)
  length = len(df)

  #Update the count dictionary
  weekly_numbers[file_name] = length

#Set the filepath
obj_to_pickle = weekly_numbers
filepath = Path(r"/content/gdrive/MyDrive/Bandit_Project")/'weekly_df_counts.pkl'

#Pickle it
with open(filepath, mode='wb') as file:
  pkl.dump(obj_to_pickle, file)

#Load it
with open(filepath, mode='rb') as file:
  obj_from_pickle = pkl.load(file)
"""
#Load already saved weekly numbers file
filepath = Path(r"/content/gdrive/MyDrive/Bandit_Project")/'weekly_df_counts.pkl'
weekly_numbers = filepath.read_bytes()

#with open(filepath, mode='rb') as file:
#  obj_from_pickle = pkl.load(file)

#Pprint the loaded object
print("[INFO] Printing the object... ")
pprint(weekly_numbers)

'[INFO] Working on df_2019_27.csv...'
'[INFO] Working on df_2019_28.csv...'
'[INFO] Working on df_2019_29.csv...'
'[INFO] Working on df_2019_30.csv...'
'[INFO] Working on df_2019_31.csv...'
'[INFO] Working on df_2019_32.csv...'
'[INFO] Working on df_2019_33.csv...'
'[INFO] Working on df_2019_34.csv...'
'[INFO] Working on df_2019_35.csv...'
'[INFO] Working on df_2019_36.csv...'
'[INFO] Working on df_2019_37.csv...'
'[INFO] Working on df_2019_38.csv...'
'[INFO] Working on df_2019_39.csv...'
'[INFO] Working on df_2019_40.csv...'
'[INFO] Working on df_2019_41.csv...'
'[INFO] Working on df_2019_42.csv...'
'[INFO] Working on df_2019_43.csv...'
'[INFO] Working on df_2019_44.csv...'
'[INFO] Working on df_2019_45.csv...'
'[INFO] Working on df_2019_46.csv...'
'[INFO] Working on df_2019_47.csv...'
'[INFO] Working on df_2019_48.csv...'
'[INFO] Working on df_2019_49.csv...'
'[INFO] Working on df_2019_50.csv...'
'[INFO] Working on df_2019_51.csv...'
'[INFO] Working on df_2019_52.csv...'
'[INFO] Work

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'[INFO] Working on df_2020_02.csv...'
'[INFO] Working on df_2020_03.csv...'
'[INFO] Working on df_2020_04.csv...'
'[INFO] Working on df_2020_05.csv...'
'[INFO] Working on df_2020_06.csv...'
'[INFO] Working on df_2020_07.csv...'
'[INFO] Working on df_2020_08.csv...'
'[INFO] Working on df_2020_09.csv...'
'[INFO] Working on df_2020_10.csv...'
'[INFO] Working on df_2020_11.csv...'
'[INFO] Working on df_2020_12.csv...'
'[INFO] Working on df_2020_13.csv...'
'[INFO] Working on df_2020_14.csv...'
'[INFO] Working on df_2020_15.csv...'
'[INFO] Working on df_2020_16.csv...'
'[INFO] Working on df_2020_17.csv...'
'[INFO] Working on df_2020_18.csv...'
'[INFO] Working on df_2020_19.csv...'
'[INFO] Working on df_2020_20.csv...'
'[INFO] Working on df_2020_21.csv...'
'[INFO] Working on df_2020_22.csv...'
'[INFO] Working on df_2020_23.csv...'
'[INFO] Working on df_2020_24.csv...'
'[INFO] Working on df_2020_25.csv...'
'[INFO] Working on df_2020_26.csv...'
'[INFO] Working on df_2020_27.csv...'
'[INFO] Work

#Loading Baseline Model

In [7]:
from space_bandits import load_model
linear_model = load_model('/content/gdrive/MyDrive/Bandit_Project/models/900_bandit')
print(type(linear_model))

<class 'space_bandits.linear.LinearBandits'>
